## This notebook explores fuel types and the volumes of cargo transfered by fuel type

This notebook requires an activation of the `monte_carlo` environment setup 
located in `analysis-rachael/env`
```
conda activate monte_carlo
```

The newest versions of the monte-carlo spills files can be found on Salish at the following location:
`/ocean/rmueller/MIDOSS/spill_files_TLPcorrected`

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml
#from monte_carlo_utils import decimal_divide
from pathlib import Path

### User Inputs

In [6]:
#~~~~~  User inputs ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Dept. of Ecology data files
ECY_dir  = Path('/Users/rmueller/Data/MIDOSS/DeptOfEcology/')
ECY_2018_xlsx = ECY_dir/'MuellerTrans4-30-20.xlsx'
ECY_2021_xlsx = ECY_dir/'MuellerTrans_5-26-21.xlsx'

# Voyage dataset
CamSheet = '/Users/rmueller/Library/Mobile Documents/com~apple~CloudDocs/Documents/'\
    'Publications/MIDOSS/MIDOSS_MuellerEtAl_paper1/Methods/origin-destination/'\
    'Origin_Destination_Analysis_updated.xlsx'

# 10,000 monte carlo spills
monte_carlo_csv = '/Users/rmueller/Data/MIDOSS/monte_carlo/SalishSeaOilSpills_fixbarge_10000.csv'

# location of output excel spreadsheets
output_directory = Path('/Users/rmueller/Data/MIDOSS/DeptOfEcology/')
graphics_directory = Path(
    '/Users/rmueller/Projects/MIDOSS/graphics_figures/monte_carlo'
)

# The precision used to calculate oil type weights 
# Trial and error showed that a precision of 2 is neccessary for weights
# to sum to 1.0
precision = 2

# unit conversions
gal2m3 = 0.00378541

# facility names

# The following list includes facilities used in Casey's origin/destination 
# analysis with names matching the Dept. of Ecology (ECY) database.  
# For example, the shapefile "Maxum Petroleum - Harbor Island Terminal" is 
# labeled as 'Maxum (Rainer Petroleum)' in the ECY database.  I use the 
# Ecology language here and will need to translate to Shapefile speak

# Terminals that are grouped in voyage joins are removed, e.g.:
# 'Maxum (Rainer Petroleum)' and 'Shell Oil LP Seattle Distribution Terminal' 
# are represented in
#  ==>'Kinder Morgan Liquids Terminal - Harbor Island' 
# 'Nustar Energy Tacoma' => 'Phillips 66 Tacoma Terminal'
facility_names = [ 
    'BP Cherry Point Refinery', 'Shell Puget Sound Refinery', 
    'Tidewater Snake River Terminal', 'SeaPort Sound Terminal', 
    'Tesoro Vancouver Terminal', 'Phillips 66 Ferndale Refinery', 
    'Phillips 66 Tacoma Terminal', 
    'Andeavor Anacortes Refinery (formerly Tesoro)',
    'Tesoro Port Angeles Terminal','U.S. Oil & Refining',
    'Naval Air Station Whidbey Island (NASWI)',
    'NAVSUP Manchester', 'Alon Asphalt Company (Paramount Petroleum)', 
    'Kinder Morgan Liquids Terminal - Harbor Island' ,  
    'Tesoro Pasco Terminal', 'REG Grays Harbor, LLC', 
    'Tidewater Vancouver Terminal','TLP Management Services LLC (TMS)'
]

# list of facility names to query monte-carlo csv file, with:
# 1) Marathon Anacortes Refinery (formerly Tesoro) instead of Andeavor 
#    Anacortes Refinery (formerly Tesoro) 
# 2) Maxum Petroleum - Harbor Island Terminal instead of 
#    Maxum (Rainer Petroleum)
facility_names_mc = [ 
    'BP Cherry Point Refinery', 'Shell Puget Sound Refinery',
    'Tidewater Snake River Terminal', 
    'SeaPort Sound Terminal', 'Tesoro Vancouver Terminal',
    'Phillips 66 Ferndale Refinery', 'Phillips 66 Tacoma Terminal', 
    'Marathon Anacortes Refinery (formerly Tesoro)',
    'Tesoro Port Angeles Terminal','U.S. Oil & Refining',
    'Naval Air Station Whidbey Island (NASWI)',
    'NAVSUP Manchester', 'Alon Asphalt Company (Paramount Petroleum)', 
    'Kinder Morgan Liquids Terminal - Harbor Island',
    'Tesoro Pasco Terminal', 'REG Grays Harbor, LLC', 
    'Tidewater Vancouver Terminal',
    'TLP Management Services LLC (TMS)'
]

---
## Load Department of Ecology 2018 data
---

In [9]:
# Import columns are: 
#   (G) Deliverer, (H) Receiver, (O) Region, (P) Product, 
#   (Q) Quantity in Gallons, (R) Transfer Type (Fueling, Cargo, or Other)', 
#   (w) DelivererTypeDescription, (x) ReceiverTypeDescription 
#2018
ECY_df = pd.read_excel(
    ECY_2018_xlsx,
    sheet_name='Vessel Oil Transfer', 
    usecols="G,H,P,Q,R,W,X"
)
[nrows,ncols] = ECY_df.shape

### Catalogue the ECY data to match the way that we grouped marine terminals in our origin-destination analysis, due to proximity, e.g. (using abbr names in some cases):
- Maxum + Shell Oil LP + Kinder Morgan -> 'Kinder Morgan Liquids Terminal - Harbor Island'
- Nustar Tacoma
- Nustar Vancouver (?)
- Tesoro Pasco
- Tidewater Snake
- Tidewater Vancouver

In [10]:
# First, count the terminals that we are still missing in groupings
print('Nustar Tacoma: ', ECY_df.loc[
    ((ECY_df.Deliverer == 'Nustar Energy Tacoma') |
    (ECY_df.Receiver == 'Nustar Energy Tacoma')),
].Deliverer.count())
print('Tesoro Pasco Terminal', ECY_df.loc[
    ((ECY_df.Deliverer == 'Tesoro Pasco Terminal') |
    (ECY_df.Receiver == 'Tesoro Pasco Terminal')),
].Deliverer.count())
print('Tidewater Snake River Terminal', ECY_df.loc[
    ((ECY_df.Deliverer == 'Tidewater Snake River Terminal') |
    (ECY_df.Receiver == 'Tidewater Snake River Terminal')),
].Deliverer.count())
print('Tidewater Vancouver Terminal', ECY_df.loc[
    ((ECY_df.Deliverer == 'Tidewater Vancouver Terminal') |
    (ECY_df.Receiver == 'Tidewater Vancouver Terminal')),
].Deliverer.count())

Nustar Tacoma:  3
Tesoro Pasco Terminal 112
Tidewater Snake River Terminal 445
Tidewater Vancouver Terminal 64


In [11]:
# count transfers prior to re-name
MaxumECY = ECY_df.loc[
    ((ECY_df.Deliverer == 'Maxum (Rainer Petroleum)') |
    (ECY_df.Receiver == 'Maxum (Rainer Petroleum)')),
].Deliverer.count()
ShellECY = ECY_df.loc[
    ((ECY_df.Deliverer == 'Shell Oil LP Seattle Distribution Terminal') |
    (ECY_df.Receiver == 'Shell Oil LP Seattle Distribution Terminal')),
].Deliverer.count()
KinderECY = ECY_df.loc[
    ((ECY_df.Deliverer == 'Kinder Morgan Liquids Terminal - Harbor Island') |
    (ECY_df.Receiver == 'Kinder Morgan Liquids Terminal - Harbor Island')),
].Deliverer.count()
NustarECY = ECY_df.loc[
    ((ECY_df.Deliverer == 'Nustar Energy Tacoma') |
    (ECY_df.Receiver == 'Nustar Energy Tacoma')),
].Deliverer.count()
PhillipsECY = ECY_df.loc[
    ((ECY_df.Deliverer == 'Phillips 66 Tacoma Terminal') |
    (ECY_df.Receiver == 'Phillips 66 Tacoma Terminal')),
].Deliverer.count()

# housekeeping: Force one name per marine transfer site for cases of data entry mistakes
ECY_df = ECY_df.replace(
    to_replace="TLP",
    value = "TLP Management Services LLC (TMS)"
)
ECY_df = ECY_df.replace(
    to_replace = "US Oil Tacoma ",
    value = "U.S. Oil & Refining"
)

In [12]:
# rename 'Maxum (Rainer Petroleum)' and 'Shell Oil LP Seattle Distribution Terminal' 
# as 'Kinder Morgan Liquids Terminal - Harbor Island'
ECY_df['Deliverer'] = ECY_df['Deliverer'].replace(
    ['Maxum (Rainer Petroleum)',
     'Shell Oil LP Seattle Distribution Terminal'], 
    'Kinder Morgan Liquids Terminal - Harbor Island'
)
ECY_df['Receiver'] = ECY_df['Receiver'].replace(
    ['Maxum (Rainer Petroleum)','Shell Oil LP Seattle Distribution Terminal'],
    'Kinder Morgan Liquids Terminal - Harbor Island'
)

# Rename Nustar Energy Tacoma -> Phillips 66 Tacoma
ECY_df['Deliverer'] = ECY_df['Deliverer'].replace(
    ['Nustar Energy Tacoma'], 
    'Phillips 66 Tacoma Terminal'
)
ECY_df['Receiver'] = ECY_df['Receiver'].replace(
    ['Nustar Energy Tacoma'],
    'Phillips 66 Tacoma Terminal'
)
    
# recount transfers
MaxumShellKinderECY = ECY_df.loc[
    ((ECY_df.Deliverer == 'Kinder Morgan Liquids Terminal - Harbor Island') |
    (ECY_df.Receiver == 'Kinder Morgan Liquids Terminal - Harbor Island')),
].Deliverer.count()

NustarPhillipsECY = ECY_df.loc[
    ((ECY_df.Deliverer == 'Phillips 66 Tacoma Terminal') |
    (ECY_df.Receiver == 'Phillips 66 Tacoma Terminal')),
].Deliverer.count()

## Tally non-caro transfers

In [38]:
# converstions
gal2liter=3.78541
gal2m3 = 0.0037854

transfers = {}
volume = {}
# ~~~ Total cargo import and  export for all oil types ~~~
transfers['cargo'] = ECY_df.loc[
    (ECY_df.TransferType == 'Cargo') &
    ((ECY_df.Receiver.isin(facility_names)) |
     (ECY_df.Deliverer.isin(facility_names))), 
].shape[0]

# ~~~ Total non-cargo import and  export for all oil types ~~~
transfers['non-cargo'] = ECY_df.loc[
    (ECY_df.TransferType != 'Cargo') &
    ((ECY_df.Receiver.isin(facility_names)) |
     (ECY_df.Deliverer.isin(facility_names))), 
].shape[0]
volume['gallon']={}

# CARGO transfers
volume['gallon']['non-cargo'] = ECY_df.loc[
    (ECY_df.TransferType != 'Cargo') &
    ((ECY_df.Receiver.isin(facility_names)) |
     (ECY_df.Deliverer.isin(facility_names))),
    'TransferQtyInGallon'
].sum(axis=0).item()
# ALL OTHER transfers
volume['gallon']['cargo'] = ECY_df.loc[
    (ECY_df.TransferType == 'Cargo') &
    ((ECY_df.Receiver.isin(facility_names)) |
     (ECY_df.Deliverer.isin(facility_names))),
    'TransferQtyInGallon'
].sum(axis=0).item()
volume['gallon']['percent_cargo'] = 100*volume['gallon']['cargo']/(volume['gallon']['cargo']+volume['gallon']['non-cargo'])

# convert to publication metric
volume['m3']={}
volume['m3']['non-cargo']= volume['gallon']['non-cargo']*gal2m3
volume['m3']['cargo']= volume['gallon']['cargo']*gal2m3
volume['m3']['percent_cargo'] = 100*volume['m3']['cargo']/(volume['m3']['cargo']+volume['m3']['non-cargo'])

# convert to BC standard metric
volume['liter']={}
volume['liter']['non-cargo'] = volume['gallon']['non-cargo']*gal2liter
volume['liter']['cargo'] = volume['gallon']['cargo']*gal2liter
volume['liter']['percent_cargo'] = 100*volume['liter']['cargo']/(volume['liter']['cargo']+volume['liter']['non-cargo'])
print('all percents are printed to verify they are the same')
pd.DataFrame.from_dict(volume)

all percents are printed to verify they are the same


,gallon,m3,liter
non-cargo,7.391174e+07,2.797855e+05,2.797862e+08
cargo,9.991996e+09,3.782370e+07,3.782380e+10
percent_cargo,9.926572e+01,9.926572e+01,9.926572e+01


In [39]:
transfers

{'cargo': 4438, 'non-cargo': 3211}

### 99% of all transfers (by volume) to/from WA marine terminals in this study are cargo transfers

## Repeat above for all transfers

In [40]:
# converstions
gal2liter=3.78541
gal2m3 = 0.0037854

transfers_all = {}
volume_all = {}
# ~~~ Total cargo import and  export for all oil types ~~~
transfers_all['cargo'] = ECY_df.loc[
    (ECY_df.TransferType == 'Cargo')  
].shape[0] - transfers['cargo']

# ~~~ Total non-cargo import and  export for all oil types ~~~
transfers_all['non-cargo'] = ECY_df.loc[
    (ECY_df.TransferType != 'Cargo')  
].shape[0]-transfers['non-cargo']

volume_all['gallon']={}
# CARGO transfers
volume_all['gallon']['non-cargo'] = ECY_df.loc[
    (ECY_df.TransferType != 'Cargo') ,
    'TransferQtyInGallon'
].sum(axis=0).item()-volume['gallon']['non-cargo'] # subtract terminals in this study

# ALL OTHER transfers
volume_all['gallon']['cargo'] = ECY_df.loc[
    (ECY_df.TransferType == 'Cargo'),
    'TransferQtyInGallon'
].sum(axis=0).item()-volume['gallon']['cargo']# subtract terminals in this study

volume_all['gallon']['percent_cargo'] = 100*volume_all['gallon']['cargo']/(
    volume_all['gallon']['cargo']+volume_all['gallon']['non-cargo'])

# convert to publication metric
volume_all['m3']={}
volume_all['m3']['non-cargo']= volume_all['gallon']['non-cargo']*gal2m3
volume_all['m3']['cargo']= volume_all['gallon']['cargo']*gal2m3
volume_all['m3']['percent_cargo'] = 100*volume_all['m3']['cargo']/(
    volume_all['m3']['cargo']+volume_all['m3']['non-cargo'])

# convert to BC standard metric
volume_all['liter']={}
volume_all['liter']['non-cargo'] = volume_all['gallon']['non-cargo']*gal2liter
volume_all['liter']['cargo'] = volume_all['gallon']['cargo']*gal2liter
volume_all['liter']['percent_cargo'] = 100*volume_all['liter']['cargo']/(
    volume_all['liter']['cargo']+volume_all['liter']['non-cargo'])
print('all percents are printed to verify they are the same')
pd.DataFrame.from_dict(volume_all)

all percents are printed to verify they are the same


,gallon,m3,liter
non-cargo,6.334694e+08,2.397935e+06,2.397941e+09
cargo,2.096002e+08,7.934205e+05,7.934226e+08
percent_cargo,2.486155e+01,2.486155e+01,2.486155e+01


In [41]:
transfers_all

{'cargo': 194, 'non-cargo': 6867}

# All transfers

In [42]:
# converstions
gal2liter=3.78541
gal2m3 = 0.0037854

transfers_all = {}
volume_all = {}
# ~~~ Total cargo import and  export for all oil types ~~~
transfers_all['cargo'] = ECY_df.loc[
    (ECY_df.TransferType == 'Cargo')  
].shape[0] 

# ~~~ Total non-cargo import and  export for all oil types ~~~
transfers_all['non-cargo'] = ECY_df.loc[
    (ECY_df.TransferType != 'Cargo')  
].shape[0]

volume_all['gallon']={}
# CARGO transfers
volume_all['gallon']['non-cargo'] = ECY_df.loc[
    (ECY_df.TransferType != 'Cargo') ,
    'TransferQtyInGallon'
].sum(axis=0).item()

# ALL OTHER transfers
volume_all['gallon']['cargo'] = ECY_df.loc[
    (ECY_df.TransferType == 'Cargo'),
    'TransferQtyInGallon'
].sum(axis=0).item()

volume_all['gallon']['percent_cargo'] = 100*volume_all['gallon']['cargo']/(
    volume_all['gallon']['cargo']+volume_all['gallon']['non-cargo'])

# convert to publication metric
volume_all['m3']={}
volume_all['m3']['non-cargo']= volume_all['gallon']['non-cargo']*gal2m3
volume_all['m3']['cargo']= volume_all['gallon']['cargo']*gal2m3
volume_all['m3']['percent_cargo'] = 100*volume_all['m3']['cargo']/(
    volume_all['m3']['cargo']+volume_all['m3']['non-cargo'])

# convert to BC standard metric
volume_all['liter']={}
volume_all['liter']['non-cargo'] = volume_all['gallon']['non-cargo']*gal2liter
volume_all['liter']['cargo'] = volume_all['gallon']['cargo']*gal2liter
volume_all['liter']['percent_cargo'] = 100*volume_all['liter']['cargo']/(
    volume_all['liter']['cargo']+volume_all['liter']['non-cargo'])
print('all percents are printed to verify they are the same')
pd.DataFrame.from_dict(volume_all)

all percents are printed to verify they are the same


,gallon,m3,liter
non-cargo,7.073811e+08,2.677720e+06,2.677727e+09
cargo,1.020160e+10,3.861712e+07,3.861722e+10
percent_cargo,9.351561e+01,9.351561e+01,9.351561e+01
